# 🎬 Hệ gợi ý (Model-based CF) với ALS (Explicit)

Notebook này huấn luyện **ALS tường minh (Explicit ALS)** cho bài toán dự đoán điểm phim và xuất dự đoán theo định dạng **`Id,Score`** (bắt đầu `Id` từ 1) giống file mẫu.

**Quy trình**
1. Đọc `train.txt` / `test.txt`
2. Ánh xạ ID gốc → chỉ số liên tục (0..n-1)
3. Tính **baseline** (μ, độ lệch người dùng bᵤ, độ lệch item bᵢ) để dự phòng cold-start
4. Xây ma trận thưa user–item
5. Huấn luyện **ALS tường minh** (tối thiểu hóa sai số bình phương + L2)
6. (Tuỳ chọn) Đánh giá nhanh RMSE trên tập holdout
7. Dự đoán cho test và xuất `submission.csv`


In [171]:

import os
import numpy as np
import pandas as pd
from scipy import sparse
from pathlib import Path
from typing import Optional, Tuple

# --- Đường dẫn (sửa nếu cần) ---
TRAIN_PATH = "data/train.txt"   # ví dụ: "/kaggle/input/.../train.txt"
TEST_PATH  = "data/test.txt"    # ví dụ: "/kaggle/input/.../test.txt"
SUB_PATH   = "output/CF_ALS_submission.csv"   # nơi lưu kết quả dự đoán

print(Path(TRAIN_PATH).resolve())
print(Path(TEST_PATH).resolve())


E:\Sao lưu onedrive\not de ra truong nao\Hệ gợi ý\Project_RS_in_HUST\data\train.txt
E:\Sao lưu onedrive\not de ra truong nao\Hệ gợi ý\Project_RS_in_HUST\data\test.txt


In [172]:

# --- Đọc dữ liệu ---
df_train = pd.read_csv(
    TRAIN_PATH,
    sep=r"\s+",
    header=None,
    names=["userid","movieid","rating"],
    engine="python",
)
df_test = pd.read_csv(
    TEST_PATH,
    sep=r"\s+",
    header=None,
    names=["userid","movieid"],
    engine="python",
)

print(df_train.shape, df_test.shape)
display(df_train.head())
display(df_test.head())


(90570, 3) (9430, 2)


,userid,movieid,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


,userid,movieid
0,1,20
1,1,33
2,1,61
3,1,117
4,1,155


In [173]:

# --- Ánh xạ ID gốc → chỉ số liên tục (0..n-1), gộp cả train & test để tránh thiếu ---
all_u = pd.concat([df_train["userid"], df_test["userid"]], ignore_index=True)
all_i = pd.concat([df_train["movieid"], df_test["movieid"]], ignore_index=True)

# Danh sách ID người dùng và phim duy nhất, không trùng lặp
uid_uniques = all_u.drop_duplicates()
iid_uniques = all_i.drop_duplicates()

# Tạo mapping: userid gốc → index 0..n_users-1
uid_map = pd.Series(np.arange(len(uid_uniques), dtype=np.int32), index=uid_uniques.values)
iid_map = pd.Series(np.arange(len(iid_uniques), dtype=np.int32), index=iid_uniques.values)

df_train["u_idx"] = df_train["userid"].map(uid_map).astype(np.int32)
df_train["i_idx"] = df_train["movieid"].map(iid_map).astype(np.int32)
df_test["u_idx"]  = df_test["userid"].map(uid_map).astype(np.int32)
df_test["i_idx"]  = df_test["movieid"].map(iid_map).astype(np.int32)

n_users = uid_map.size
n_items = iid_map.size
print(f"n_users={n_users}, n_items={n_items}, train_rows={len(df_train)}, test_rows={len(df_test)}")

df_train

n_users=943, n_items=1682, train_rows=90570, test_rows=9430


,userid,movieid,rating,u_idx,i_idx
0,1,1,5,0,0
1,1,2,3,0,1
2,1,3,4,0,2
3,1,4,3,0,3
4,1,5,3,0,4
...,...,...,...,...,...
90565,943,1047,2,942,1034
90566,943,1074,4,942,1063
90567,943,1188,3,942,1173
90568,943,1228,3,942,1211


In [174]:
# --- Baseline (μ + b_u + b_i) dùng làm dự phòng (fallback) cho cold-start ---
mu = df_train["rating"].mean()
lambda_reg_bias = 10.0

In [175]:
# Ma trận tạm để tính nhanh chỉ mục người dùng/item đã có rating
R_tmp = sparse.coo_matrix(
    (df_train["rating"].astype(np.float32), (df_train["u_idx"], df_train["i_idx"])),
    shape=(n_users, n_items),
).tocsr()

print(R_tmp)
R_tmp.shape

# đếm số phần tử rỗng của ma trận R_tmp
num_zeros = R_tmp.shape[0] * R_tmp.shape[1] - R_tmp.count_nonzero()
print("Số phần tử rỗng trong ma trận R_tmp:", num_zeros)

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 90570 stored elements and shape (943, 1682)>
  Coords	Values
  (0, 0)	5.0
  (0, 1)	3.0
  (0, 2)	4.0
  (0, 3)	3.0
  (0, 4)	3.0
  (0, 5)	5.0
  (0, 6)	4.0
  (0, 7)	1.0
  (0, 8)	5.0
  (0, 9)	3.0
  (0, 10)	2.0
  (0, 11)	5.0
  (0, 12)	5.0
  (0, 13)	5.0
  (0, 14)	5.0
  (0, 15)	5.0
  (0, 16)	3.0
  (0, 17)	4.0
  (0, 18)	5.0
  (0, 19)	1.0
  (0, 20)	4.0
  (0, 21)	4.0
  (0, 22)	3.0
  (0, 23)	4.0
  (0, 24)	3.0
  :	:
  (942, 703)	1.0
  (942, 716)	4.0
  (942, 744)	4.0
  (942, 747)	2.0
  (942, 754)	4.0
  (942, 756)	3.0
  (942, 776)	2.0
  (942, 785)	3.0
  (942, 787)	3.0
  (942, 807)	4.0
  (942, 815)	4.0
  (942, 816)	3.0
  (942, 822)	2.0
  (942, 830)	4.0
  (942, 919)	5.0
  (942, 931)	1.0
  (942, 933)	5.0
  (942, 1000)	2.0
  (942, 1014)	2.0
  (942, 1031)	3.0
  (942, 1034)	2.0
  (942, 1063)	4.0
  (942, 1173)	3.0
  (942, 1211)	3.0
  (942, 1322)	3.0
Số phần tử rỗng trong ma trận R_tmp: 1495556


In [176]:
users_items = [R_tmp[u].indices for u in range(n_users)]
items_users = [R_tmp[:, i].indices for i in range(n_items)]

# những index item mà được user 0 rating
print(users_items[0])

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [177]:
b_u = np.zeros(n_users, dtype=np.float32)
b_i = np.zeros(n_items, dtype=np.float32)

print(b_u)
b_u.shape

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

(943,)

### 5.1 Ý tưởng tối ưu

Ta muốn tìm $b_u, b_i$ để minimize hàm:

$$
\sum_{(u,i)\in\Omega} \bigl(r_{ui} - \mu - b_u[u] - b_i[i]\bigr)^2
\;+\;
\lambda_{\text{reg}}\left(\sum_u b_u[u]^2 + \sum_i b_i[i]^2\right)
$$

- $\Omega$: tập $(u, i)$ có rating.  
- Thành phần thứ nhất: MSE.  
- Thành phần thứ hai: L2 penalty cho bias.

Giải analytic theo từng biến cho ta công thức dạng:

$$
b_u[u] =
\frac{\sum_{i \in I(u)} \bigl(r_{ui} - \mu - b_i[i]\bigr)}
     {\lambda_{\text{reg}} + |I(u)|}
$$

$$
b_i[i] =
\frac{\sum_{u \in U(i)} \bigl(r_{ui} - \mu - b_u[u]\bigr)}
     {\lambda_{\text{reg}} + |U(i)|}
$$

Đoạn code đang cài đặt đúng 2 công thức này, lặp đi lặp lại vài vòng (ở đây là 3 vòng).


In [178]:
# Lặp vài vòng coordinate-descent đơn giản
for _ in range(3):
    # cập nhật b_u
    for u in range(n_users):
        idx_i = users_items[u]
        if len(idx_i) == 0: 
            b_u[u] = 0.0
            continue
        r_ui = R_tmp[u, idx_i].toarray().ravel()
        b_u[u] = ((r_ui - mu - b_i[idx_i]).sum()) / (lambda_reg_bias + len(idx_i))
    # cập nhật b_i
    for i in range(n_items):
        idx_u = items_users[i]
        if len(idx_u) == 0:
            b_i[i] = 0.0
            continue
        r_ui = R_tmp[idx_u, i].toarray().ravel()
        b_i[i] = ((r_ui - mu - b_u[idx_u]).sum()) / (lambda_reg_bias + len(idx_u))

b_u

array([-0.0059783 ,  1.9346808 ,  1.572     ,  1.733602  ,  0.9079987 ,
        1.5805012 ,  2.193693  ,  1.1293321 ,  1.4269352 ,  2.1653898 ,
        1.6259413 ,  1.2751254 ,  1.5274935 ,  1.720453  ,  1.3460186 ,
        2.159719  ,  0.54750615,  1.8016396 ,  1.091826  ,  0.82311594,
        1.2144421 ,  1.2229052 ,  1.1847918 ,  1.8019462 ,  1.5205501 ,
        1.298729  ,  0.87167233,  1.5420411 ,  1.6367933 ,  1.4171916 ,
        1.4926043 ,  0.8168363 ,  1.7328886 ,  1.5601104 ,  1.3690418 ,
        1.6564534 ,  1.1326145 ,  2.049456  ,  1.4020021 ,  1.1648128 ,
        0.8314652 ,  1.7849333 ,  1.8761282 ,  1.0245376 ,  1.4418856 ,
        1.384251  ,  1.5020763 ,  1.6175071 ,  0.7175505 ,  1.1575571 ,
        0.38121003,  2.062832  ,  0.5799624 ,  1.7034711 ,  0.42959425,
        1.4752035 ,  1.6076592 ,  1.5321833 ,  2.1470947 ,  2.0427697 ,
        0.80776626,  1.2217851 ,  1.1217667 ,  1.2036076 ,  1.3733238 ,
        1.4536133 ,  1.3009233 ,  0.9071681 ,  1.338723  ,  1.26

- Dự đoán rating cho user $u$, item $i$ **chỉ bằng baseline**:

  $$
  \hat r^{\text{baseline}}_{ui} = \mu + b_u[u] + b_i[i]
  $$

- `np.clip` để giữ trong khoảng $[1, 5]$ (vì rating thường là $1$–$5$).
- Đây là mô hình **rất đơn giản** nhưng hiệu quả khá tốt và được dùng:
  - Làm baseline để so sánh với ALS / MF / NeuMF.
  - Làm **fallback cho cold-start**:
    - Khi user hoặc item chưa có latent vector (U/V) hoặc quá ít rating.
    - Hoặc nếu dot-product $U_u \cdot V_i$ có vấn đề $\rightarrow$ fallback về baseline.

In [179]:
def predict_baseline(u: int, i: int, clip: Optional[Tuple[float,float]]=(1,5)) -> float:
    """Dự đoán theo baseline: μ + b_u + b_i (có clip về [1,5] nếu cần)."""
    val = mu + b_u[u] + b_i[i]
    if clip is not None:
        val = float(np.clip(val, clip[0], clip[1]))
    return float(val)

print("Baseline sẵn sàng. Global mean:", mu)

Baseline sẵn sàng. Global mean: 3.5238268742409184


In [180]:
# --- Tách nhanh một tập holdout để ước lượng RMSE (tuỳ chọn) ---
rng = np.random.default_rng(2024)
mask = rng.random(len(df_train)) < 0.1  # lấy 10% làm holdout
df_valid = df_train[mask].reset_index(drop=True)
df_tr    = df_train[~mask].reset_index(drop=True)
print("Số dòng train dùng ALS:", df_tr.shape, "| holdout:", df_valid.shape)

# --- Xây ma trận thưa CSR từ df_tr ---
R = sparse.coo_matrix(
    (df_tr["rating"].astype(np.float32), (df_tr["u_idx"], df_tr["i_idx"])),
    shape=(n_users, n_items),
).tocsr()

# Tuỳ chọn: huấn luyện trên dữ liệu đã trừ đi trung bình toàn cục
CENTER = True
r_mean = mu


Số dòng train dùng ALS: (81526, 5) | holdout: (9044, 5)


# Cài đặt ALS tường minh (Explicit ALS)

In [181]:
import numpy as np

V = np.array([
    [1.0, 2.0],   # item 0
    [3.0, 4.0],   # item 1
    [5.0, 6.0],   # item 2
])
print("V =\n", V)
print("shape V:", V.shape)   # (3, 2)

VtV = V.T @ V
print("V.T @ V =\n", VtV)
print("shape V.T @ V:", VtV.shape)   # (2, 2)

V =
 [[1. 2.]
 [3. 4.]
 [5. 6.]]
shape V: (3, 2)
V.T @ V =
 [[35. 44.]
 [44. 56.]]
shape V.T @ V: (2, 2)


In [182]:
rng = np.random.default_rng(42)
U = 0.1 * rng.standard_normal((n_users, 6)).astype(np.float32)
U.shape

(943, 6)

In [183]:
def als_explicit_with_user_bias(R_csr: sparse.csr_matrix, 
                                n_users: int, n_items: int, 
                                k: int = 64,          # số chiều latent "thật" cho w,h
                                reg: float = 0.1, 
                                n_iters: int = 10,
                                center: bool = True,
                                mean_val: float = 0.0,
                                random_state: int = 42):
    """
    ALS tường minh với user bias:
    r_ui - mean_val ≈ w_u^T h_i + b_u
    Trong đó:
      - U[u] = [w_u, b_u] ∈ R^{k+1}
      - V[i] = [h_i, 1]   ∈ R^{k+1}
    """
    rng = np.random.default_rng(random_state)
    
    k_ext = k + 1  # thêm 1 chiều cho bias

    # Khởi tạo U, V mở rộng
    U = 0.1 * rng.standard_normal((n_users, k_ext)).astype(np.float32)    # (n_users, k+1)
    V = 0.1 * rng.standard_normal((n_items, k_ext)).astype(np.float32)    # (n_items, k+1)

    # Chiều cuối của V là 1 (vector h̃_i có phần tử cuối = 1)
    V[:, -1] = 1.0

    # Danh sách chỉ mục để truy cập nhanh
    users_items = [R_csr[u].indices for u in range(n_users)]
    items_users = [R_csr[:, i].indices for i in range(n_items)]

    I_ext = np.eye(k_ext, dtype=np.float32)

    for it in range(n_iters):
        # --- Cập nhật U (cố định V) ---
        # (ALS explicit chuẩn: với mỗi user u, giải
        #  (V_I^T V_I + reg I) U_u = V_I^T r_u )
        for u in range(n_users):
            idx_i = users_items[u]
            if len(idx_i) == 0:
                continue
            V_i = V[idx_i]                              # (m, k+1)
            r_u = R_csr[u, idx_i].toarray().ravel()     # (m,)
            if center:
                r_u = r_u - mean_val

            A = V_i.T @ V_i + reg * I_ext              # (k+1, k+1)
            b = V_i.T @ r_u                            # (k+1,)
            U[u] = np.linalg.solve(A, b)

        # --- Cập nhật V (cố định U) ---
        for i in range(n_items):
            idx_u = items_users[i]
            if len(idx_u) == 0:
                continue
            U_u = U[idx_u]                              # (m, k+1)
            r_i = R_csr[idx_u, i].toarray().ravel()     # (m,)
            if center:
                r_i = r_i - mean_val

            A = U_u.T @ U_u + reg * I_ext              # (k+1, k+1)
            b = U_u.T @ r_i                            # (k+1,)
            V[i] = np.linalg.solve(A, b)

            # Giữ cố định chiều bias của item = 1 (h̃_i = [h_i, 1])
            V[i, -1] = 1.0

        print(f"Hoàn tất vòng lặp ALS {it+1}/{n_iters}.")

    return U, V

In [184]:

# --- Huấn luyện ALS ---
K = 128         # số chiều ẩn
LAMBDA = 0.1   # hệ số L2
N_ITERS = 10   # số vòng lặp ALS

U, V = als_explicit_with_user_bias(R, n_users, n_items, k=K, reg=LAMBDA, n_iters=N_ITERS, center=CENTER, mean_val=r_mean, random_state=2025)
print("Kích thước nhân tố:", U.shape, V.shape)


Hoàn tất vòng lặp ALS 1/10.
Hoàn tất vòng lặp ALS 2/10.
Hoàn tất vòng lặp ALS 3/10.
Hoàn tất vòng lặp ALS 4/10.
Hoàn tất vòng lặp ALS 5/10.
Hoàn tất vòng lặp ALS 6/10.
Hoàn tất vòng lặp ALS 7/10.
Hoàn tất vòng lặp ALS 8/10.
Hoàn tất vòng lặp ALS 9/10.
Hoàn tất vòng lặp ALS 10/10.
Kích thước nhân tố: (943, 129) (1682, 129)


In [185]:

# --- Ước lượng RMSE trên tập holdout (tuỳ chọn) ---
def predict_als_with_user_bias(u, i, U, V, center=True, mean_val=0.0, clip=(1, 5)):
    """
    Dự đoán rating cho cặp (u, i) với mô hình có user bias.
    U[u] = [w_u, b_u], V[i] = [h_i, 1]
    """
    val = float(U[u] @ V[i])      # = w_u^T h_i + b_u
    if center:
        val += mean_val           # cộng lại μ
    if clip is not None:
        val = float(np.clip(val, clip[0], clip[1]))
    return val


def rmse_holdout(df_valid, U, V, clip=(1,5), center=True, mean_val=0.0):
    se = 0.0
    n = 0
    for u, i, r in df_valid[["u_idx","i_idx","rating"]].itertuples(index=False):
        pred = predict_als_with_user_bias(u, i, U, V, center=center, mean_val=mean_val)
        if clip is not None:
            pred = np.clip(pred, clip[0], clip[1])
        se += (pred - r) ** 2
        n += 1
    return np.sqrt(se / max(n, 1))

val_rmse = rmse_holdout(df_valid, U, V, clip=(1,5), center=CENTER, mean_val=r_mean)
print(f"Holdout RMSE: {val_rmse:.5f}")


Holdout RMSE: 1.03899


In [186]:

# --- Dự đoán cho test kèm fallback về baseline nếu gặp cold-start ---
def predict_with_fallback(u, i, U, V, clip=(1,5)) -> float:
    has_u = np.any(U[u]) if (0 <= u < U.shape[0]) else False
    has_i = np.any(V[i]) if (0 <= i < V.shape[0]) else False
    if has_u and has_i:
        val = float(U[u] @ V[i])
        if CENTER:
            val += r_mean
        if clip is not None:
            val = float(np.clip(val, clip[0], clip[1]))
        return val
    # Nếu thiếu user/item trong nhân tố, quay về baseline
    return predict_baseline(u, i, clip=clip)

CLIP = (1, 5)
preds = np.empty(len(df_test), dtype=np.float32)
for idx, (u, i) in enumerate(zip(df_test["u_idx"].values, df_test["i_idx"].values)):
    preds[idx] = predict_with_fallback(u, i, U, V, clip=CLIP)

print("Ví dụ 10 dự đoán đầu:", preds[:10])


Ví dụ 10 dự đoán đầu: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [187]:

# --- Xuất file submission theo định dạng mẫu (Id bắt đầu từ 1) ---
submission = pd.DataFrame({
    "Id": np.arange(1, len(df_test) + 1, dtype=np.int64),
    "Score": preds
})
submission.to_csv(SUB_PATH, index=False, float_format="%.6f")
print("Đã lưu:", SUB_PATH, submission.shape)
display(submission.head())


Đã lưu: output/CF_ALS_submission.csv (9430, 2)


,Id,Score
0,1,1.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,1.0
